# **Model Context Protocol**

### **What is MCP?**
MCP is a standardised way to integrate tools with your applications. As defined by Anthropic:
> **MCP is an open protocol that standardizes how your LLM Applicaitons connect to and work with your tools and data sources.**

### **The Problem**
Since we have been using LangChain, we know this pain intimately.
Imagine you are building an AI assistant for your company. You want it to access:
- PostgreSQL (for customer data).
- Slack (to read internal chats).
- GitHub (to look at code).

**Without MCP:** We have to find (or write) a specific Python library for Postgres, another for Slack, and another for GitHub. We then have to wrap each one in a LangChain Tool.
- Imagine there are 10 different AI applications (Host) and 100 different data sources, developers have to build 1,000 different integrations. It is unscalable.

**Another Example:** Imagine you have 3 favorite AI models: Claude, ChatGPT, and Gemini. And you have 3 data sources: Google Drive, Slack, and GitHub. Without MCP, if you want Claude to talk to all three, you have to write 3 specific integrations. If you want Gemini to talk to them, you have to write 3 different integrations. If you want ChatGPT to talk to them... you get the point.

### **The Pain Points**
1. Wasted Effort: Developers are rebuilding the exact same "Google Drive Connector" for every different AI app.
2. More Maintanance: If Google Drive API changes, developer has to do changes in each of the "M" integrations.

### **The Solution: The "USB-C" Analogy**
Think of MCP exactly like a USB-C port.
- Before USB-C: You had a separate charger for your phone, your laptop, your camera, and your headphones. It was messy.
- With USB-C: You have one standard port. If you buy a hard drive with a USB-C connector, you know it will work with your MacBook, your Windows PC, or your iPad. The hard drive manufacturer doesn't need to know which computer you own.

**With MCP:** You build a "Google Drive MCP Server" once.
- Your LangChain Agent can plug into it.
- Claude Desktop can plug into it.
- Cursor/Windsurf IDE can plug into it.

The **"Server"** (the hard drive) doesn't care who the **"Host"** (the computer) is. It just speaks the standard language.

**Important Note:** Once you write an MCP Server, it works in LangChain, but it also works everywhere else.

### **Core Architecture**
This is the most critical part for a developer. In the MCP world, there are three distinct roles.
1. **The Host (The "Interface")**
- **What it is:** This is the application the user interacts with.
- **Example:** In your case, your LangChain Python script is the Host. Other examples include the Claude Desktop app or an IDE like VS Code (with an MCP extension).
- **Role:** It decides when to call a tool or ask for context. It holds the LLM key.

2. **The Client (The "Connector")**
- **What it is:** This is a hidden software component that lives inside the Host.
- **Example:** langchain-mcp-adapters provides the Client logic.
- **Role:** It maintains the 1:1 connection with the Server. It handles the handshake and sends messages back and forth. You rarely write this from scratch; you just use a library.

3. **The Server (The "Toolkit")**
- **What it is:** A standalone process (script) that exposes **tools**, **resources (i.e. read only data)** and **prompts**.
- **Example:** A Python script running fastmcp that connects to a SQLite database.
- **Role:** It waits for commands. It has no AI. It is dumb but capable. It says, "I have a tool called get_customer_id. If you give me a name, I will return an ID."

### **The Transport Layer: How Client and Server talk**
The Client and Server need a wire to communicate. MCP supports two main types of "wires."
1. **Stdio (Standard Input/Output) -> Local**
- **How it works:** The Host actually spawns the Server process directly on the same machine. They talk by printing text to the console (stdin/stdout).
- **Analogy:** A distinct neurological connection. The brain (Host) is directly hardwired to the hand (Server).
- **Pros:** Extremely fast, very secure (data never leaves your machine), easiest to set up.
- **Cons:** Both must be on the same computer.
- **Your Use Case:** We will start here. Your LangChain script will run a generic Python subprocess to talk to your tools.

2. **SSE (Server-Sent Events) -> Remote/Web**
- **How it works:** The Server runs on a distant cloud machine (like AWS or Render). The Host connects via HTTP.
- **Analogy:** A Walkie-Talkie. The Brain is in one building, the Hand is in another. They talk over the airwaves.
- **Pros:** You can share one Server with your whole team.
- **Cons:** Requires authentication, network management, and is slightly more complex to deploy.

## **Under the Hood**

### **JSON-RPC 2.0: The Language**
MCP doesn't invent a new format; it uses the standard JSON-RPC 2.0 specification. Every message is a JSON object with a specific structure.
- **Request:** Has an `id` and `method`
- **Response:** Has the same `id` and `result`
- **Notification:** Has `method` but **no id**. No reply expected.

```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": { ... }
}
```

### **The Handshake: Establishing Client and Server Connection**
Before any tools can be used, the Client and Server must agree on a version and capabilities. This happens automatically when the process starts.

**Step A:Client (LangChain) says Hello** 
- The Client sends an initialize request. It tells the Server what protocol version it supports and what it can do.

```JSON
// CLIENT -> SERVER
{
  "jsonrpc": "2.0",
  "id": 0,
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "roots": { "listChanged": true }, // "I can handle file system roots"
      "sampling": {} // "I can handle LLM sampling/generation requests"
    },
    "clientInfo": { "name": "LangChainClient", "version": "1.0" }
  }
}
```

**Step B: Server Responds** 
- The Server replies with its own capabilities. This is crucial. If the Server says "I don't support resources," the Client knows not to ask for them.

```JSON
// SERVER -> CLIENT
{
  "jsonrpc": "2.0",
  "id": 0,
  "result": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "tools": {},      // "I have tools"
      "resources": {},  // "I have resources"
      "prompts": {}     // "I have prompts"
    },
    "serverInfo": { "name": "my-weather-server", "version": "0.1" }
  }
}
```

**Step C: Client Acknowledges The Client sends a notification saying, "Okay, we are initialized."**

```JSON
// CLIENT -> SERVER
{
  "jsonrpc": "2.0",
  "method": "notifications/initialized"
}
```

### **The Three Primitives (Resources, Tools and Prompts)**
This is the most important conceptual part. MCP divides the world into three specific types of interactions.

1. **Resources (Read Only Data)**
- **Concept:** Think of this as Data retrieval. It corresponds to a `GET request`.
- **Behavior:** The Client (LLM) asks to "read" something. It is strictly read-only.
- **Use Case:** Reading a file, fetching the current rows of a database, reading a log file.
- **LangChain Equivalent:** This maps directly to LangChain's `DocumentLoader`.
- **The Raw Call:**
```JSON
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "resources/read",
  "params": {
    "uri": "file:///application/logs/error.txt"
  }
}
```

2. **Tools (Active Capabilities)**
- **Concept:** Think of this as Function Calling. It corresponds to a `POST request`.
- **Behavior:** The Client asks the Server to perform a calculation or an action.
- **Use Case:** "Calculate sum," "Query API," "Git Commit."
- **LangChain Equivalent:** This maps directly to LangChain's `@tool` or `BaseTool`.
- **The Raw Call:**

```JSON
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "get_weather",
    "arguments": {
      "city": "Hyderabad"
    }
  }
}
```

3. **Prompts (Reusable Templates)**
- **Concept:** This is often the most confusing for beginners. These are Server-defined prompt templates.
- **Behavior:** The Server stores a prompt structure (like a System Prompt) that the User can select.
- **Use Case:**
    - A "Git Server" might expose a prompt called code_review that automatically pre-fills the prompt with "Please review the following code changes from the git diff...".
    - This saves the user from typing "Please act as a code reviewer..." every time.
- **LangChain Equivalent:** This maps directly to LangChain's `PromptTemplate`.

## **Hello World**

### **Why FastMCP?**
The official Python SDK for MCP is powerful but verbose—it requires setting up async loops, stream handlers, and complex initialization.

**FastMCP is a wrapper (inspired by FastAPI)** that handles all the protocol plumbing for you.
- **Official SDK:** 50+ lines of code to hello world.
- **Using FastMCP:** 5 lines of code.

It automatically generates the JSON Schema for your tools based on your Python type hints (str, int, etc.), which the LLM uses to understand how to call your functions.

### **Installation**
```python
! pip install fastmcp
```

### **Implementing a Server**

We will build a simple Weather Server. To keep it simple, we will mock the data so you don't need a real API key yet.

Let's now create a file named `weather_server.py`:

In [ ]:
from fastmcp import FastMCP

# Step 1: Initialize the Server
mcp = FastMCP("My Local Weather Server")

# --- DATA (Mock Database) ---
weather_data = {
    "Hyderabad": "Sunny, 28°C",
    "London": "Rainy, 12°C",
    "San Francisco": "Foggy, 16°C"
}

# --- TOOL: Active Capability (The "POST" request) ---
@mcp.tool()
def get_forecast(city: str) -> str:
    """
    Get the current weather forecast for a specific city.
    args:
        city: The name of the city (e.g. 'Hyderabad', 'London')
    """
    # Simulate an API call
    result = weather_data.get(city, "Unknown City")
    return f"The weather in {city} is currently: {result}"

# --- RESOURCE: Passive Context (The "GET" request) ---
@mcp.resource("weather://alerts")
def get_weather_alerts() -> str:
    """Returns a list of active severe weather alerts."""
    return "WARNING: Heatwave approaching Hyderabad. Heavy rain expected in London."

# 4. Run the server
if __name__ == "__main__":
    mcp.run()